In [2]:
import numpy as np
from tqdm import trange

In [3]:
# load data from video
import cv2
import torchvision
import torchvision.transforms

# resolution
W = 1920//2
H = 1080//2

def get_video_data(path):
    frames = []
    
    cap = cv2.VideoCapture(data_path+video_name) 
    count = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (W,H))
            if count % 30 == 0:
                frames.append(frame)
                cv2.imshow('frame', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            count += 1
        else:
            break
    cap.release()
    cv2.destroyAllWindows()
    
    return np.array(frames)

#data_path = "data/videos/labeled/"
data_path = "data/videos/with_crossroads/"# CHANGE THIS
video_name = "production ID_4370833.mp4"  # CHANGE THIS

frames = get_video_data(data_path+video_name)

array([[[[238, 226, 212],
         [238, 226, 212],
         [238, 226, 212],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[239, 227, 213],
         [237, 225, 211],
         [237, 225, 211],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[242, 230, 216],
         [239, 227, 213],
         [238, 226, 212],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        ...,

        [[ 82,  78,  81],
         [ 84,  80,  83],
         [ 85,  81,  84],
         ...,
         [102, 101, 102],
         [103, 102, 104],
         [103, 102, 104]],

        [[ 85,  81,  84],
         [ 87,  83,  86],
         [ 87,  83,  86],
         ...,
         [103, 102, 104],
         [103, 102, 104],
         [104, 103, 105]],

        [[ 87,  83,  86],
         [ 87,  83,  86],
         [ 87,  83,  86],
         ...,
         [103, 102, 104],
        

In [5]:
frames.shape

(71, 540, 960, 3)

In [6]:
# create model
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # image size
        self.W = 1920//2
        self.H = 1080//2

        # Convolutional layers
        # TODO: decide layers architecture
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        # Convolutional layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)   # output is binary (is or is not crossroad)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [7]:
# train
model = Net()